<a href="https://colab.research.google.com/github/ShreyNaik123/virtuorigin/blob/main/Virtuorigin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 18aOUuJVV6kIgpnqKoP-AW3cY6ZxhP9Aa

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download dataclusterlabs/indian-vehicle-dataset

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=18aOUuJVV6kIgpnqKoP-AW3cY6ZxhP9Aa
To: /content/kaggle.json
100% 68.0/68.0 [00:00<00:00, 367kB/s]
100% 2.37G/2.37G [00:18<00:00, 103MB/s]
100% 2.37G/2.37G [00:18<00:00, 139MB/s]


In [ ]:
from zipfile import ZipFile
zip_ref = ZipFile('/content/indian-vehicle-dataset.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os

In [ ]:
base_path = os.getcwd()
annotations_path = "/content/annotation/annotation"
images_path = "/content/Vehicle_5_classes_sample"

In [ ]:
if(not os.path.exists('data')):
  os.mkdir('data')
  os.mkdir('data/train')
  os.mkdir('data/test')
  os.mkdir('data/val')

In [ ]:
data_path = '/content/data'

In [ ]:
# fixing the annotations format
import os
import xml.etree.ElementTree as ET


class_mapping = {
    'tempo': 0,
    'bus': 1,
    'vehicle_truck':2,
    'tractor':3,
    'car':4,
    'two_wheelers':5,
    'bicycle':6,
    'auto':7,}
def convert_to_yolo(xml_files, yolo_annotation_dir):

  for xml_path in xml_files:

      tree = ET.parse(xml_path)
      root = tree.getroot()


      width = int(root.find('size/width').text)
      height = int(root.find('size/height').text)


      yolo_file = xml_path.split(".")[-2].split('/')[-1] + '.txt'
      yolo_path = os.path.join(yolo_annotation_dir, yolo_file)

      with open(yolo_path, 'w') as yolo_annotation:
          for obj in root.findall('object'):
              class_name = obj.find('name').text
              if class_name not in class_mapping:
                  continue

              class_id = class_mapping[class_name]

              xmin = float(obj.find('bndbox/xmin').text)
              ymin = float(obj.find('bndbox/ymin').text)
              xmax = float(obj.find('bndbox/xmax').text)
              ymax = float(obj.find('bndbox/ymax').text)


              x_center = (xmin + xmax) / 2 / width
              y_center = (ymin + ymax) / 2 / height
              box_width = (xmax - xmin) / width
              box_height = (ymax - ymin) / height


              yolo_annotation.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")


In [ ]:
# rename folders to match the annotation folder names
for c in os.listdir(images_path):
  original_name = os.path.join(images_path, c)
  new_name = os.path.join(images_path, c.split(" ")[0].lower())
  os.rename(original_name, new_name)

In [ ]:
for dir in os.listdir(data_path):
  path = os.path.join(data_path, dir)
  os.mkdir(path + "/labels")
  os.mkdir(path + "/images")

In [ ]:
import random

def shuffle_two_arrays(image_paths, annotation_paths):
  shuffled_paths = list(zip(image_paths, annotation_paths))

  random.shuffle(shuffled_paths)

  shuffled_image_paths, shuffled_annotation_paths = zip(*shuffled_paths)

  return shuffled_image_paths, shuffled_annotation_paths

In [ ]:


def split_dataset(images_path,annotation_path, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):

  image_files = []
  annotation_files=[]

  for classes in os.listdir(images_path):
      path = os.path.join(images_path, classes)
      for image in os.listdir(path):
        image_path = os.path.join(path, image)
        image_files.append(image_path)

  for classes in os.listdir(annotation_path):
    path = os.path.join(annotation_path, classes)
    for image in os.listdir(path):
      image_path = os.path.join(path, image)
      annotation_files.append(image_path)


  image_files, annotation_files = shuffle_two_arrays(sorted(image_files), sorted(annotation_files))
  # # Split the image and XML files into train, test, and validation sets.
  train_images = []
  train_xmls = []
  val_images = []
  val_xmls = []
  test_images = []
  test_xmls = []

  print(len(image_files))
  print(len(annotation_files))
  for i in range(len(image_files)):
    random_number = random.random()

    if random_number < train_ratio:
      train_images.append(image_files[i])
      train_xmls.append(annotation_files[i])
    elif random_number < train_ratio + val_ratio:
      val_images.append(image_files[i])
      val_xmls.append(annotation_files[i])
    else:
      test_images.append(image_files[i])
      test_xmls.append(annotation_files[i])

  return train_images, train_xmls, val_images, val_xmls, test_images, test_xmls


In [ ]:
train_images, train_xmls, val_images, val_xmls, test_images, test_xmls = split_dataset(images_path, annotations_path)

744
744


In [ ]:
test_xmls[0], test_images[0]

('/content/annotation/annotation/tractor/20210601_19_22_35_000_5AKEXwOfciZh1Rfxj4ahQeFIb1p1_T_4000_3000.xml',
 '/content/Vehicle_5_classes_sample/tractor/20210601_19_22_35_000_5AKEXwOfciZh1Rfxj4ahQeFIb1p1_T_4000_3000.jpg')

In [ ]:
import numpy as np
np.intersect1d(train_images, val_images)

array([], dtype='<U108')

In [ ]:
train_images_dir = '/content/data/train/images'
train_xmls_dir = '/content/data/train/labels'
test_images_dir = '/content/data/test/images'
test_xmls_dir = '/content/data/test/labels'
val_images_dir = '/content/data/val/images'
val_xmls_dir = '/content/data/val/labels'


In [ ]:
convert_to_yolo(train_xmls, train_xmls_dir)

In [ ]:
convert_to_yolo(test_xmls, test_xmls_dir)
convert_to_yolo(val_xmls, val_xmls_dir)

In [ ]:
import shutil
for image_file in train_images:
  shutil.copy(image_file, train_images_dir)

for image_file in val_images:
  shutil.copy(image_file, val_images_dir)

for image_file in test_images:
  shutil.copy(image_file, test_images_dir)


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 13.6 MB/s eta 0:00:00


In [ ]:
import os
import xml.etree.ElementTree as ET


def get_names_from_xml_files(folder_path):


  names = set()

  for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):
      filepath = os.path.join(folder_path, filename)

      tree = ET.parse(filepath)
      root = tree.getroot()
      for object_element in root.findall("object"):

        name_element = object_element.find("name")

        name = name_element.text
        names.add(name)

  return list(names)


In [ ]:
import yaml

def create_yolo_yaml(data_file_path, class_names, train_path, val_path, test_path, nc, names):
    yolo_data = {
        'train': train_path,
        'val': val_path,
        'test': test_path,
        'nc': nc,         # Number of classes
        'names': names,   # Class names
    }

    with open(data_file_path, 'w') as outfile:
        yaml.dump(yolo_data, outfile, default_flow_style=False)

In [ ]:
data_file_path = '/content/data/data.yaml'
class_names = get_names_from_xml_files('/content/data/train/labels')
train_path = '/content/data/train'
val_path = '/content/data/val'
test_path = '/content/data/test'
nc = len(class_names)
names = class_names

In [ ]:
create_yolo_yaml(data_file_path, class_names, train_path, val_path, test_path, nc, names)

In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data=/content/data/data.yaml \
epochs=100 \
imgsz=640

100% 21.5M/21.5M [00:00<00:00, 221MB/s]
Ultralytics YOLOv8.0.203 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/data/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True,